In [1]:
import pandas as pd
import numpy as np

In [2]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\solar_no_mastr_geo.csv"
solar_no_mastr = pd.read_csv(data, sep=";", encoding="utf-8")
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\pv_incl_geotags_50hz.csv"
solar_geotagged = pd.read_csv(data, low_memory=False)
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\stations.csv"
weather_stations = pd.read_csv(data)

In [3]:
weather_stations.reset_index(inplace=True)

In [4]:
weather_stations = weather_stations.iloc[:, 1:]

In [12]:
weather_stations.to_csv("weather_stations_reset.csv", sep=";", encoding="utf-8")

In [13]:
solar_geotagged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205286 entries, 0 to 205285
Data columns (total 20 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   EinheitMastrNummer                        205286 non-null  object 
 1   Bundesland                                205286 non-null  object 
 2   Landkreis                                 205286 non-null  object 
 3   Gemeinde                                  205285 non-null  object 
 4   Postleitzahl                              205286 non-null  int64  
 5   Ort                                       205286 non-null  object 
 6   Bruttoleistung                            205286 non-null  float64
 7   Nettonennleistung                         205286 non-null  float64
 8   Leistungsbegrenzung                       205286 non-null  object 
 9   EinheitlicheAusrichtungUndNeigungswinkel  201886 non-null  float64
 10  Hauptausrichtung    

In [14]:
solar_no_mastr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29213 entries, 0 to 29212
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EEG_Anlagenschlüssel   29213 non-null  object 
 1   EEG_Mastr_Nr           0 non-null      float64
 2   PLZ                    29213 non-null  int64  
 3   Bundesland             29213 non-null  object 
 4   Installierte_Leistung  29213 non-null  object 
 5   Energieträger          29213 non-null  object 
 6   Leistungsmessung       29213 non-null  object 
 7   Regelbarkeit           29213 non-null  object 
 8   Inbetriebnahme         29213 non-null  object 
 9   Außerbetriebnahme      30 non-null     object 
 10  lat                    29189 non-null  float64
 11  lon                    29189 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 2.7+ MB


In [18]:
!pip install haversine

In [5]:
import haversine as hs

In [16]:
solar_geo_prep_1 = solar_geotagged.iloc[:, [15, 14]]

In [6]:
solar_geo_prep_2 = solar_no_mastr.iloc[:, [10, 11]]

In [7]:
weather_prep = weather_stations.iloc[:, [2,3]]

In [8]:
weather_prep

,lat,lon
0,50.7913,12.8720
1,51.1278,13.7543
2,50.4283,12.9536
3,51.1621,14.9506
4,53.6332,9.9881
5,51.3932,10.3123
6,51.3151,12.4462
7,51.4347,12.2396
8,52.2085,14.1180
9,52.3812,13.0622


In [21]:
def distances(i):
    geoloc = (solar_geo_prep_1.iloc[i, 0], solar_geo_prep_1.iloc[i, 1])
    station_no = []
    geo_distances = []
    for n in range(len(weather_prep)):
        station_geo = (weather_prep.iloc[n, 0], weather_prep.iloc[n, 1])
        station_no.append(n)
        distance = hs.haversine(geoloc, station_geo)
        geo_distances.append(distance)
        geo_df = pd.DataFrame({"station_no": station_no, "distance": geo_distances})
    return geo_df[geo_df.distance == geo_df.distance.min()]



In [22]:
station_df = []
for i in range(len(solar_geo_prep_1)):
    station_df.append(distances(i))

station_assignement = pd.concat(station_df).reset_index().loc[:, ["station_no", "distance"]]

In [23]:
station_assignement

,station_no,distance
0,12,34.510722
1,8,46.283741
2,11,30.930478
3,5,6.828936
4,12,58.317932
...,...,...
205281,0,24.229873
205282,10,74.655993
205283,11,84.425497
205284,1,20.182744


In [24]:
solar_mastr_stations = solar_geotagged.merge(station_assignement, left_index=True, right_index=True)

In [25]:
solar_mastr_stations.to_csv("solar_mastr_station_assigned.csv", encoding="utf-8", sep=";", index=False)

In [9]:
def distances_2(i):
    geoloc = (solar_geo_prep_2.iloc[i, 0], solar_geo_prep_2.iloc[i, 1])
    station_no = []
    geo_distances = []
    for n in range(len(weather_prep)):
        station_geo = (weather_prep.iloc[n, 0], weather_prep.iloc[n, 1])
        station_no.append(n)
        distance = hs.haversine(geoloc, station_geo)
        geo_distances.append(distance)
        geo_df = pd.DataFrame({"station_no": station_no, "distance": geo_distances})
    return geo_df[geo_df.distance == geo_df.distance.min()]


In [10]:
station_df_2 = []
for i in range(len(solar_geo_prep_2)):
    station_df_2.append(distances_2(i))

geo_df_2 = pd.concat(station_df_2)

In [11]:
geo_df_2 = geo_df_2.reset_index()

In [12]:
geo_df_2.iloc[:, -2:]

,station_no,distance
0,9,11.832075
1,9,11.832075
2,9,11.832075
3,9,4.948227
4,9,4.948227
...,...,...
29184,10,44.034743
29185,10,44.034743
29186,5,73.616740
29187,8,6.918748


In [13]:
solar_no_mastr_station = solar_no_mastr.merge(geo_df_2, left_index=True, right_index=True)

In [14]:
solar_no_mastr_station

,EEG_Anlagenschlüssel,EEG_Mastr_Nr,PLZ,Bundesland,Installierte_Leistung,Energieträger,Leistungsmessung,Regelbarkeit,Inbetriebnahme,Außerbetriebnahme,lat,lon,index,station_no,distance
0,E41860010000000000602161970900001,NaN,14476,Brandenburg,"5,4",Solar,Nein,70 % Begrenzung,2008-11-14,NaN,52.474970,12.979712,9,9,11.832075
1,E41860010000000000602162132700001,NaN,14476,Brandenburg,"4,5",Solar,Nein,70 % Begrenzung,2008-12-05,NaN,52.474970,12.979712,9,9,11.832075
2,E41694010000000000004038447600001,NaN,14476,Brandenburg,"7,8",Solar,Nein,70 % Begrenzung,2014-08-28,NaN,52.474970,12.979712,9,9,11.832075
3,E41694010000000000414021671800001,NaN,14469,Brandenburg,"3,72",Solar,Nein,70 % Begrenzung,2019-03-25,NaN,52.423078,13.037531,9,9,4.948227
4,E41694010000000000004035525800001,NaN,14469,Brandenburg,"4,35",Solar,Nein,nicht regelbar,2011-06-29,NaN,52.423078,13.037531,9,9,4.948227
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29184,E41725010000000000000000000000033,NaN,39261,Sachsen-Anhalt,"6,84",Solar,Nein,nicht regelbar,2009-12-04,NaN,51.969740,12.074546,10,10,44.034743
29185,E40543010009520304403233605400001,NaN,3044,Brandenburg,"2,55",Solar,Nein,nicht regelbar,2011-12-09,NaN,51.781532,14.330684,10,10,44.034743
29186,E40543010009520304626819005000001,NaN,3046,Brandenburg,"6,58",Solar,Nein,nicht regelbar,2010-09-29,NaN,51.760072,14.310221,5,5,73.616740
29187,E40543010009520304629359008000001,NaN,3046,Brandenburg,"4,6",Solar,Nein,nicht regelbar,2008-09-19,NaN,51.760072,14.310221,8,8,6.918748


In [83]:
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\wind_50hz_no_mastr.csv"
wind_no_mastr = pd.read_csv(data, sep=";")
data = r"C:\Users\danie\Desktop\WBS.DataScience_Bootcamp\final_project\wind_50hz_mastr_merged.csv"
wind_mastr = pd.read_csv(data, sep=";")

In [85]:
wind_no_mastr_geo = wind_no_mastr.copy().iloc[:, -2:]

In [86]:
wind_no_mastr_geo

,latitude,longitude
0,52.578180,12.872044
1,53.293714,14.011771
2,53.396101,13.959544
3,53.405539,14.191801
4,53.405539,14.191801
...,...,...
1074,51.412160,13.447317
1075,53.152869,14.057674
1076,53.152869,14.057674
1077,53.152869,14.057674


In [87]:
def distances_3(i):
    geoloc = (wind_no_mastr_geo.iloc[i, 0], wind_no_mastr_geo.iloc[i, 1])
    station_no = []
    geo_distances = []
    for n in range(len(weather_prep)):
        station_geo = (weather_prep.iloc[n, 0], weather_prep.iloc[n, 1])
        station_no.append(n)
        distance = hs.haversine(geoloc, station_geo)
        geo_distances.append(distance)
        geo_df = pd.DataFrame({"station_no": station_no, "distance": geo_distances})
    return geo_df[geo_df.distance == geo_df.distance.min()]

In [88]:
station_df_3 = []
for i in range(len(wind_no_mastr_geo)):
    station_df_3.append(distances_3(i))

geo_df_3 = pd.concat(station_df_3)

In [89]:
geo_df_3 = geo_df_3.reset_index().iloc[:, -2:]

In [94]:
geo_df_3

,station_no,distance
0,9,25.408486
1,12,50.271753
2,12,39.418417
3,12,38.541898
4,12,38.541898
...,...,...
1074,1,38.156147
1075,12,65.791368
1076,12,65.791368
1077,12,65.791368


In [95]:
wind_no_mastr

,EEG_Anlagenschlüssel,EEG_Mastr_Nr,Straße_Flurstück,PLZ,Ort_Gemarkung,Bundesland,Installierte_Leistung,Energieträger,Leistungsmessung,Inbetriebnahme,Außerbetriebnahme,Netzabgang,latitude,longitude
0,E40450010225600000000007819670001,NaN,"Flur 1, Flurstück 39",14641,Markee,Brandenburg,2300.0,Wind an Land,Ja,2008-09-09,NaN,2021-12-31,52.578180,12.872044
1,E40450010000000000000021907000185,NaN,"Grünow, 2/175",17291,Grünow,Brandenburg,3450.0,Wind an Land,Ja,2017-09-15,NaN,NaN,53.293714,14.011771
2,E40450010000000000000082140900138,NaN,"Tornow, 1, 179",17291,Tornow,Brandenburg,2000.0,Wind an Land,Ja,2009-08-06,NaN,NaN,53.396101,13.959544
3,E40450010000000000000082067000092,NaN,"Woddow, 5, 238",17326,Brüssow,Brandenburg,2000.0,Wind an Land,Ja,2008-03-14,NaN,NaN,53.405539,14.191801
4,E40450010000000000000082062000085,NaN,"Wollschow, 1, 264",17326,Brüssow,Brandenburg,2000.0,Wind an Land,Ja,2007-03-22,NaN,NaN,53.405539,14.191801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,E41874010000000000350000000900002,NaN,Friedrich-Schiller-Str. 10 Y,1609,Gröditz,Sachsen,3450.0,Wind an Land,Ja,2016-06-23,NaN,NaN,51.412160,13.447317
1075,E41860010000000000602127889400001,NaN,"Hohengüstrow, Flur 3/ 12",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-01,NaN,NaN,53.152869,14.057674
1076,E41860010000000000602127889400002,NaN,"Hohengüstrow, Flur 3/ 14",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-31,NaN,NaN,53.152869,14.057674
1077,E41860010000000000602127889400003,NaN,"Hohengüstrow, Flur 3/ 19",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-31,NaN,NaN,53.152869,14.057674


In [99]:
wind_no_mastr_stations = wind_no_mastr.merge(geo_df_3, left_index=True, right_index=True)

In [100]:
wind_no_mastr_stations

,EEG_Anlagenschlüssel,EEG_Mastr_Nr,Straße_Flurstück,PLZ,Ort_Gemarkung,Bundesland,Installierte_Leistung,Energieträger,Leistungsmessung,Inbetriebnahme,Außerbetriebnahme,Netzabgang,latitude,longitude,station_no,distance
0,E40450010225600000000007819670001,NaN,"Flur 1, Flurstück 39",14641,Markee,Brandenburg,2300.0,Wind an Land,Ja,2008-09-09,NaN,2021-12-31,52.578180,12.872044,9,25.408486
1,E40450010000000000000021907000185,NaN,"Grünow, 2/175",17291,Grünow,Brandenburg,3450.0,Wind an Land,Ja,2017-09-15,NaN,NaN,53.293714,14.011771,12,50.271753
2,E40450010000000000000082140900138,NaN,"Tornow, 1, 179",17291,Tornow,Brandenburg,2000.0,Wind an Land,Ja,2009-08-06,NaN,NaN,53.396101,13.959544,12,39.418417
3,E40450010000000000000082067000092,NaN,"Woddow, 5, 238",17326,Brüssow,Brandenburg,2000.0,Wind an Land,Ja,2008-03-14,NaN,NaN,53.405539,14.191801,12,38.541898
4,E40450010000000000000082062000085,NaN,"Wollschow, 1, 264",17326,Brüssow,Brandenburg,2000.0,Wind an Land,Ja,2007-03-22,NaN,NaN,53.405539,14.191801,12,38.541898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,E41874010000000000350000000900002,NaN,Friedrich-Schiller-Str. 10 Y,1609,Gröditz,Sachsen,3450.0,Wind an Land,Ja,2016-06-23,NaN,NaN,51.412160,13.447317,1,38.156147
1075,E41860010000000000602127889400001,NaN,"Hohengüstrow, Flur 3/ 12",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-01,NaN,NaN,53.152869,14.057674,12,65.791368
1076,E41860010000000000602127889400002,NaN,"Hohengüstrow, Flur 3/ 14",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-31,NaN,NaN,53.152869,14.057674,12,65.791368
1077,E41860010000000000602127889400003,NaN,"Hohengüstrow, Flur 3/ 19",16306,Briest,Brandenburg,1500.0,Wind an Land,Ja,2002-12-31,NaN,NaN,53.152869,14.057674,12,65.791368


In [101]:
wind_mastr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10562 entries, 0 to 10561
Data columns (total 22 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   EegInbetriebnahmedatum                    10562 non-null  object 
 1   EegMaStRNummer                            10562 non-null  object 
 2   AnlagenschluesselEeg                      10247 non-null  object 
 3   InstallierteLeistung                      10562 non-null  float64
 4   VerhaeltnisReferenzertragErtrag5Jahre_nv  10562 non-null  int64  
 5   AusschreibungZuschlag                     10294 non-null  float64
 6   AnlageBetriebsstatus                      10562 non-null  object 
 7   Bundesland                                10562 non-null  object 
 8   Landkreis                                 10331 non-null  object 
 9   Gemeinde                                  10331 non-null  object 
 10  Postleitzahl                      

In [102]:
wind_mastr_geo = wind_mastr.iloc[:, [13, 12]]

In [103]:
wind_mastr_geo

,Breitengrad,Laengengrad
0,53.883993,13.426895
1,52.176389,11.465000
2,52.602453,13.789959
3,51.892620,13.490910
4,52.508658,11.200092
...,...,...
10557,51.031705,13.153049
10558,52.022600,12.670200
10559,51.685345,11.474370
10560,50.492964,11.596373


In [104]:
def distances_4(i):
    geoloc = (wind_mastr_geo.iloc[i, 0], wind_mastr_geo.iloc[i, 1])
    station_no = []
    geo_distances = []
    for n in range(len(weather_prep)):
        station_geo = (weather_prep.iloc[n, 0], weather_prep.iloc[n, 1])
        station_no.append(n)
        distance = hs.haversine(geoloc, station_geo)
        geo_distances.append(distance)
        geo_df = pd.DataFrame({"station_no": station_no, "distance": geo_distances})
    return geo_df[geo_df.distance == geo_df.distance.min()]

In [105]:
station_df_4 = []
for i in range(len(wind_mastr_geo)):
    station_df_4.append(distances_4(i))

geo_df_4 = pd.concat(station_df_4)

In [107]:
geo_df_4 = geo_df_4.reset_index().iloc[:, -2:]

In [109]:
wind_mastr_station = wind_mastr.merge(geo_df_4, left_index=True, right_index=True)

In [110]:
wind_mastr_station

,EegInbetriebnahmedatum,EegMaStRNummer,AnlagenschluesselEeg,InstallierteLeistung,VerhaeltnisReferenzertragErtrag5Jahre_nv,AusschreibungZuschlag,AnlageBetriebsstatus,Bundesland,Landkreis,Gemeinde,...,Nettonennleistung,Technologie,Nabenhoehe,Rotordurchmesser,AuflageAbschaltungLeistungsbegrenzung,DatumEndgueltigeStilllegung,DatumBeginnVoruebergehendeStilllegung,DatumWiederaufnahmeBetrieb,station_no,distance
0,2013-06-13,EEG985279937212,E4186001EA01000000000025566200001,3050.0,0,0.0,In Betrieb,Mecklenburg-Vorpommern,Vorpommern-Greifswald,Neetzow-Liepen,...,3050.0,Horizontalläufer,99.0,101.0,1.0,NaN,NaN,NaN,12,44.966486
1,2010-12-16,EEG929161321895,E4187802S120200000000009523600001,2500.0,0,0.0,In Betrieb,Sachsen-Anhalt,Börde,Hohe Börde,...,2500.0,Horizontalläufer,100.0,100.0,0.0,NaN,NaN,NaN,11,81.464016
2,2004-12-01,EEG910993068881,E41860010000000000602147220400001,2000.0,0,0.0,In Betrieb,Brandenburg,Märkisch-Oderland,Altlandsberg,...,2000.0,Horizontalläufer,105.0,90.0,1.0,NaN,NaN,NaN,8,49.133767
3,2017-03-18,EEG972555160839,E4186001EA01000000000082827500005,2530.0,0,0.0,In Betrieb,Brandenburg,Teltow-Fläming,Dahmetal,...,2530.0,Horizontalläufer,120.0,120.0,1.0,NaN,NaN,NaN,8,55.429873
4,2015-01-16,EEG944856859491,E4187802EA01000000000069094900001,2300.0,0,0.0,In Betrieb,Sachsen-Anhalt,Altmarkkreis Salzwedel,Gardelegen,...,2300.0,Horizontalläufer,98.0,82.0,1.0,NaN,NaN,NaN,11,55.515166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10557,2001-09-12,EEG982109012149,E41081010000000000000000000008232,1500.0,1,0.0,In Betrieb,Sachsen,Mittelsachsen,Striegistal,...,1500.0,Horizontalläufer,63.0,70.0,0.0,NaN,NaN,NaN,0,33.209313
10558,1995-02-22,EEG923286374538,E41860010000000000602125720500001,500.0,1,0.0,In Betrieb,Brandenburg,Potsdam-Mittelmark,Rabenstein/Fläming,...,500.0,Horizontalläufer,65.0,43.0,0.0,NaN,NaN,NaN,9,47.996157
10559,1999-11-03,EEG938652786068,E41081010000000000000000000013704,660.0,1,0.0,In Betrieb,Sachsen-Anhalt,Harz,Falkenstein/Harz,...,660.0,Horizontalläufer,65.0,47.0,0.0,NaN,NaN,NaN,7,59.792211
10560,1994-09-11,EEG989708738579,E4183201RS50557347000064951300001,600.0,1,0.0,In Betrieb,Thüringen,Saale-Orla-Kreis,Remptendorf,...,600.0,Horizontalläufer,45.0,43.0,0.0,NaN,NaN,NaN,0,95.872294


In [114]:
wind_no_mastr_stations.to_csv("wind_no_mastr_station_assigned.csv", encoding="utf-8", sep=";", index=False)
wind_mastr_station.to_csv("wind_mastr_station_assigned.csv", encoding="utf-8", sep=";", index=False)

In [115]:

solar_mastr_stations.to_csv("solar_mastr_station_assigned.csv", encoding="utf-8", sep=";", index=False)

In [ ]:
solar_no_mastr_station.to_csv("solar_no_mastr_station_assigned.csv", encoding="utf-8", sep=";", index=False)